In [ ]:
import pandas as pd

In [ ]:
# return model specific errors as a df, and the full dataset df
def get_dfs(error_file_path, full_input_df_path):
    
    return pd.read_csv(error_file_path, index_col=[0]), pd.read_csv(full_input_df_path)

In [ ]:
# change to the path of the error sheet
error_df, full_df = get_dfs('~/Downloads/flan-t5-xl.csv',
                            '../data/ingestion_tmp_data/fact_checking_full_input_information_3_20_23.csv')

In [ ]:
full_df.head()

In [ ]:
error_df.head()

In [ ]:
def get_model_stem(model_fact):
    extra_id_start_index = model_fact.index(' <extra_')
    assert(extra_id_start_index != -1)
    return model_fact[:extra_id_start_index]

In [ ]:
def get_dataset_id(model_fact, df_stems, full_df):
    model_stem = get_model_stem(model_fact).strip()
    # print(f"model fact - {model_fact}")
    # print(f"model stem - {model_stem}")
    for s in df_stems:
        if model_stem == s:
            return list(full_df[full_df['stem'] == s].dataset_id)[0]
    # fail case for stuff we've already gotten rid of during the dedupe/paraphrase-dropping process
    # print(f"Could not find original in dataset for {model_fact} -- that's ok!")
    return -1

In [ ]:
wrong_facts = []
wrong_ids = []
model = []
# change to whatever you're using
model_name = "t5-xl"
# optional to append a 'todo' automatically
resolution = []
for model_fact in list(error_df['facts']):
    original_dataset_id = get_dataset_id(model_fact, list(full_df['stem']), full_df)
    
    if original_dataset_id == -1:
        continue
        
    row = full_df[full_df['dataset_id'] == original_dataset_id]
    wrong_facts.append(list(row['stem'])[0] + " " + list(row['false'])[0] + " " + list(row['true'])[0])
    wrong_ids.append(original_dataset_id)
    model.append(model_name)
    resolution.append("todo")

In [ ]:
error_df_subset = pd.DataFrame({'model': model, 'dataset_id': wrong_ids, 'resolution': resolution, 'wrong_fact_with_true_false': wrong_facts})

In [ ]:
error_df_subset.head()

In [ ]:
# optional to put this file wherever on your local
error_df_subset.to_csv('~/Downloads/t5_errors.csv', index=False)